In [136]:
!pip install nltk

In [177]:
import pandas as pd
import numpy as np
from string import punctuation
import re

In [178]:
essays = pd.read_csv('essay_dataset.csv', encoding = "utf-8")
essays.head()

,ID,EssayTopic,EssayBody,ParagraphCount,WordCount,EssayScore,CC_Score,LR_Score,GRA_Score,TA_Score
0,1,Increasing the price of petrol is the best way...,"Over the last century, our cities faced unprec...",4,350,6.0,5.5,5.5,6.5,6.5
1,2,Fatherhood ought to be emphasized as much as m...,"According to some people, the role of a father...",4,419,6.0,5.5,5.5,6.5,7.0
2,3,Best way to reduce the number of crime among y...,"Thesedays, number of crime commit by the young...",3,197,5.0,5.5,5.0,6.5,5.0
3,4,Some people believe that it is good to share a...,Some people believe that sharing details of re...,4,281,6.0,6.5,5.5,6.5,6.0
4,5,Some people believe that teaching children at ...,People these days have contradictory opinion r...,12,410,5.5,5.5,5.0,5.5,5.0


In [179]:
essays.shape

(24254, 10)

In [180]:
essays['EssayTopic'] = essays['EssayTopic'].apply(lambda x: x.encode('ascii', 'ignore').\
                                          strip().decode())

essays['EssayBody'] = essays['EssayBody'].apply(lambda x: x.encode('ascii', 'ignore').\
                                          strip().decode())


In [181]:
isnull = essays.isnull().sum()
dtype = essays.dtypes
print('Number of null values in each column:\n \n{}\n'.format(isnull))
print('Data Type of each column:\n \n{} \n'.format(dtype))

Number of null values in each column:
 
ID                0
EssayTopic        0
EssayBody         0
ParagraphCount    0
WordCount         0
EssayScore        0
CC_Score          0
LR_Score          0
GRA_Score         0
TA_Score          0
dtype: int64

Data Type of each column:
 
ID                  int64
EssayTopic         object
EssayBody          object
ParagraphCount      int64
WordCount           int64
EssayScore        float64
CC_Score          float64
LR_Score          float64
GRA_Score         float64
TA_Score          float64
dtype: object 



In [182]:

essays.head()

,ID,EssayTopic,EssayBody,ParagraphCount,WordCount,EssayScore,CC_Score,LR_Score,GRA_Score,TA_Score
0,1,Increasing the price of petrol is the best way...,"Over the last century, our cities faced unprec...",4,350,6.0,5.5,5.5,6.5,6.5
1,2,Fatherhood ought to be emphasized as much as m...,"According to some people, the role of a father...",4,419,6.0,5.5,5.5,6.5,7.0
2,3,Best way to reduce the number of crime among y...,"Thesedays, number of crime commit by the young...",3,197,5.0,5.5,5.0,6.5,5.0
3,4,Some people believe that it is good to share a...,Some people believe that sharing details of re...,4,281,6.0,6.5,5.5,6.5,6.0
4,5,Some people believe that teaching children at ...,People these days have contradictory opinion r...,12,410,5.5,5.5,5.0,5.5,5.0


In [183]:
def clean_topic(text):
    text = text.lower()  # Lowercase text
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    return text


In [184]:
essays['EssayTopic'] = essays['EssayTopic'].map(clean_topic)

In [185]:
essays.head()

,ID,EssayTopic,EssayBody,ParagraphCount,WordCount,EssayScore,CC_Score,LR_Score,GRA_Score,TA_Score
0,1,increasing the price of petrol is the best way...,"Over the last century, our cities faced unprec...",4,350,6.0,5.5,5.5,6.5,6.5
1,2,fatherhood ought to be emphasized as much as m...,"According to some people, the role of a father...",4,419,6.0,5.5,5.5,6.5,7.0
2,3,best way to reduce the number of crime among y...,"Thesedays, number of crime commit by the young...",3,197,5.0,5.5,5.0,6.5,5.0
3,4,some people believe that it is good to share a...,Some people believe that sharing details of re...,4,281,6.0,6.5,5.5,6.5,6.0
4,5,some people believe that teaching children at ...,People these days have contradictory opinion r...,12,410,5.5,5.5,5.0,5.5,5.0


In [189]:
a = essays['EssayTopic'][2]
a

'best way to reduce the number of crime among young generation is to teach parents good parenting skills at what extent do you agree disagree give your personal exampl'